In [0]:
dbutils.secrets.listScopes()

In [0]:
# --- Configuration ---
# Key Vault & Secrets
kv_scope_name = "intuitive-scope" 
client_id_key = "service-principal-client-id"
client_secret_key = "service-principal-client-secret"
tenant_id_key = "service-principal-tenant-id"

# ADLS Configuration
adls_account_name = "intuitivedl" 
adls_filesystem_raw = "raw"
adls_filesystem_processed = "processed"
adls_filesystem_presentation = "presentation"
adls_mount_point_raw = f"/mnt/intuitivedl/{adls_filesystem_raw}"
adls_mount_point_processed = f"/mnt/intuitivedl/{adls_filesystem_processed}"
adls_mount_point_presentation = f"/mnt/intuitivedl/{adls_filesystem_presentation}"

In [0]:
# DBFS Paths 
raw_path = adls_mount_point_raw
processed_path = adls_mount_point_processed
gold_path = adls_mount_point_presentation # Using 'gold' conceptually for presentation layer

# --- Fetch Secrets ---
try:
    client_id = dbutils.secrets.get(scope=kv_scope_name, key=client_id_key)
    client_secret = dbutils.secrets.get(scope=kv_scope_name, key=client_secret_key)
    tenant_id = dbutils.secrets.get(scope=kv_scope_name, key=tenant_id_key)
    print("Successfully retrieved secrets from Key Vault.")
except Exception as e:
    print(f"ERROR: Failed to retrieve secrets from Key Vault scope '{kv_scope_name}'. Check scope and key names.")
    print(e)

In [0]:
# --- Spark Configuration for ADLS using Service Principal ---
configs = {
"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": client_id,
"fs.azure.account.oauth2.client.secret": client_secret,
"fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"
}
print("Spark configuration set for ADLS access.")

In [0]:
# --- Mount ADLS Containers ---
def mount_adls(filesystem_name, mount_point):
    adls_path = f"abfss://{filesystem_name}@{adls_account_name}.dfs.core.windows.net/"
    print(f"Attempting to mount {adls_path} to {mount_point}...")

    # Check if already mounted, unmount if necessary (makes it idempotent)
    if any(mount.mountPoint == mount_point for mount in dbutils.fs.mounts()):
        print(f"Mount point {mount_point} already exists. Unmounting first...")
        try:
            dbutils.fs.unmount(mount_point)
            print(f"Successfully unmounted {mount_point}.")
        except Exception as e:
            # Unmount might fail if in use, log warning but proceed with mount attempt
            print(f"Warning: Failed to unmount {mount_point} (maybe in use?): {e}")

    # Mount the container
    try:
        dbutils.fs.mount(
            source=adls_path,
            mount_point=mount_point,
            extra_configs=configs
        )
        print(f"Successfully mounted {filesystem_name} to {mount_point}")
    except Exception as e:
        # Check if the error is because it's already mounted by another process 
        if "Directory already mounted" in str(e):
             print(f"Mount point {mount_point} appears to be already mounted (possibly by another process). Assuming OK.")
        else:
            print(f"ERROR: Failed to mount {mount_point}: {e}")
            # dbutils.notebook.exit(f"Mount failed for {mount_point}") 

# Mount all zones
mount_adls(adls_filesystem_raw, adls_mount_point_raw)
mount_adls(adls_filesystem_processed, adls_mount_point_processed)
mount_adls(adls_filesystem_presentation, adls_mount_point_presentation)

# --- Verify Mounts ---
print("\nCurrent Mounts:")
display(dbutils.fs.mounts())

In [0]:
print(f"\nRaw Path: {raw_path}")
print(f"Processed Path (Silver): {processed_path}")
print(f"Presentation Path (Gold): {gold_path}")